In [ ]:
import matplotlib
matplotlib.use("Agg")
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.applications import ResNet152V2
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os

Using TensorFlow backend.


In [ ]:
def plot_training(H, N):
	# construct a plot that plots and saves the training history
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
	plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
	plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
	plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
	plt.title("Training Loss and Accuracy")
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend(loc="lower left")
	#plt.savefig(plotPath)

In [ ]:
trainPath = '/content/drive/My Drive/COVID-19/data/train'
testPath = '/content/drive/My Drive/COVID-19/data/test'

totalTrain = len(list(paths.list_images(trainPath)))
totalTest = len(list(paths.list_images(testPath)))

In [ ]:
print(totalTrain)
print(totalTest)

255
64


In [ ]:
trainAug = ImageDataGenerator(
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

testAug = ImageDataGenerator()

In [ ]:
BATCH_SIZE = 32
trainGen = trainAug.flow_from_directory(
	trainPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size=BATCH_SIZE)

testGen = testAug.flow_from_directory(
	testPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=BATCH_SIZE)

Found 255 images belonging to 2 classes.
Found 64 images belonging to 2 classes.


In [ ]:
baseModel = ResNet152V2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))


headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

234553344/234545216 [==============================] - 4s 0us/step


In [ ]:
for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

In [ ]:
H = model.fit_generator(
	trainGen,
	steps_per_epoch=totalTrain // BATCH_SIZE,
	validation_data=testGen,
	validation_steps=totalTest // BATCH_SIZE,
	epochs=50)

Epoch 1/50
7/7 [==============================] - 174s 25s/step - loss: 1.3478 - accuracy: 0.5381 - val_loss: 6.7100 - val_accuracy: 0.6250
Epoch 2/50
7/7 [==============================] - 141s 20s/step - loss: 0.5983 - accuracy: 0.7399 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 3/50
7/7 [==============================] - 139s 20s/step - loss: 0.2795 - accuracy: 0.8879 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 4/50
7/7 [==============================] - 142s 20s/step - loss: 0.2079 - accuracy: 0.9152 - val_loss: 0.9586 - val_accuracy: 0.4844
Epoch 5/50
7/7 [==============================] - 139s 20s/step - loss: 0.2909 - accuracy: 0.9193 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 6/50
7/7 [==============================] - 139s 20s/step - loss: 0.1270 - accuracy: 0.9686 - val_loss: 0.0000e+00 - val_accuracy: 0.5000
Epoch 7/50
7/7 [==============================] - 139s 20s/step - loss: 0.1312 - accuracy: 0.9507 - val_loss: 0.0000e+00 - val_accuracy: 0.5000


In [ ]:
print("[INFO] evaluating after fine-tuning network head...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // config.BATCH_SIZE))
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))
plot_training(H, 50, config.WARMUP_PLOT_PATH)

[INFO] evaluating after fine-tuning network head...


NameError: ignored

In [ ]:
# reset our data generators
trainGen.reset()
testGen.reset()
# now that the head FC layers have been trained/initialized, lets
# unfreeze the final set of CONV layers and make them trainable
for layer in baseModel.layers[15:]:
	layer.trainable = True
# loop over the layers in the model and show which ones are trainable
# or not
for layer in baseModel.layers:
	print("{}: {}".format(layer, layer.trainable))

In [ ]:
print("[INFO] re-compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the model again, this time fine-tuning *both* the final set
# of CONV layers along with our set of FC layers
H = model.fit_generator(
	trainGen,
	steps_per_epoch=totalTrain // BATCH_SIZE,
	validation_data=testGen,
	validation_steps=totalTest // BATCH_SIZE,
	epochs=20)

In [ ]:
print("[INFO] evaluating after fine-tuning network...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // BATCH_SIZE))
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))
plot_training(H, 20)